In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies= pd.read_csv('movies.csv')
credit= pd.read_csv('credits.csv')

In [3]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26-10-2015,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",16-07-2012,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",07-03-2012,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
print('The Movie dataframe has {} rows and {} columns'.format(movies.shape[0],movies.shape[1]))

The Movie dataframe has 4803 rows and 20 columns


In [5]:
credit.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [6]:
print('The Credit dataframe has {} rows and {} columns'.format(credit.shape[0],credit.shape[1]))

The Credit dataframe has 4803 rows and 4 columns


In [7]:
df= movies.merge(credit, on='title')

In [8]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26-10-2015,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",16-07-2012,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",07-03-2012,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [9]:
print('The Merged dataframe consisting of movies and credit  has {} rows and {} columns'.format(df.shape[0],df.shape[1]))

The Merged dataframe consisting of movies and credit  has 4808 rows and 23 columns


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4808 entries, 0 to 4807
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4808 non-null   int64  
 1   genres                4808 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4808 non-null   int64  
 4   keywords              4808 non-null   object 
 5   original_language     4808 non-null   object 
 6   original_title        4808 non-null   object 
 7   overview              4805 non-null   object 
 8   popularity            4808 non-null   float64
 9   production_companies  4808 non-null   object 
 10  production_countries  4808 non-null   object 
 11  release_date          4807 non-null   object 
 12  revenue               4808 non-null   int64  
 13  runtime               4806 non-null   float64
 14  spoken_languages      4808 non-null   object 
 15  status               

# TOP MOVIES BASED ON WEIGHTED RATING

In [11]:
top_movies = df[['movie_id','title','release_date','vote_count','vote_average','popularity','genres']]

In [12]:
top_movies.head()

,movie_id,title,release_date,vote_count,vote_average,popularity,genres
0,19995,Avatar,10-12-2009,11800,7.2,150.437577,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,285,Pirates of the Caribbean: At World's End,19-05-2007,4500,6.9,139.082615,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."
2,206647,Spectre,26-10-2015,4466,6.3,107.376788,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
3,49026,The Dark Knight Rises,16-07-2012,9106,7.6,112.312950,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam..."
4,49529,John Carter,07-03-2012,2124,6.1,43.926995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."


In [13]:
top_movies['year'] = pd.to_datetime(top_movies['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [14]:
top_movies.head()

,movie_id,title,release_date,vote_count,vote_average,popularity,genres,year
0,19995,Avatar,10-12-2009,11800,7.2,150.437577,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2009
1,285,Pirates of the Caribbean: At World's End,19-05-2007,4500,6.9,139.082615,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",2007
2,206647,Spectre,26-10-2015,4466,6.3,107.376788,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2015
3,49026,The Dark Knight Rises,16-07-2012,9106,7.6,112.312950,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",2012
4,49529,John Carter,07-03-2012,2124,6.1,43.926995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2012


I will use the IMBD rating formula to come up with the Top Movies Chart

Weighted Rating(WR)=$\frac{R(v)}{(v+m)}$ + $\frac{C(m)}{(v+m)}$

Where 
- v= the number of votes for the movie
- m= minimum number of votes required to be listed 
- R= averate rating for the movie
- C= mean vote across the whole movie
      

In [15]:
average_of_votes= top_movies[top_movies['vote_average'].notnull()]['vote_average']
vote_counts= top_movies[top_movies['vote_count'].notnull()]['vote_count']
C = top_movies[top_movies['vote_average'].notnull()]['vote_average'].mean()
m = vote_counts.quantile(0.90)


In [16]:
def weighted_rating(x):
    v =x['vote_count']
    R =x['vote_average']
    return (v/(v+m)*R)+(m/(v+m)*C)

In [17]:
top_movies['wr']=top_movies.apply(weighted_rating,axis=1)

# TOP MOVIES

In [18]:
top_movies= top_movies.sort_values('wr',ascending=False).head(100)
top_movies

,movie_id,title,release_date,vote_count,vote_average,popularity,genres,year,wr
1887,278,The Shawshank Redemption,23-09-1994,8205,8.5,136.747729,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...",1994,8.058418
662,550,Fight Club,15-10-1999,9413,8.3,146.757391,"[{""id"": 18, ""name"": ""Drama""}]",1999,7.938559
65,155,The Dark Knight,16-07-2008,12002,8.2,187.322927,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...",2008,7.919460
3237,680,Pulp Fiction,08-10-1994,8428,8.3,121.463076,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 80, ""n...",1994,7.903898
96,27205,Inception,14-07-2010,13752,8.1,167.583710,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",2010,7.862760
3342,238,The Godfather,14-03-1972,5893,8.4,143.659698,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...",1972,7.850278
95,157336,Interstellar,05-11-2014,10867,8.1,724.247784,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 18, ""...",2014,7.808911
809,13,Forrest Gump,06-07-1994,7927,8.2,138.133331,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",1994,7.802453
329,122,The Lord of the Rings: The Return of the King,01-12-2003,8064,8.1,123.630332,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",2003,7.726556
1996,1891,The Empire Strikes Back,17-05-1980,5879,8.2,78.517830,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...",1980,7.697021


In [19]:
mov = df[['movie_id','title','release_date','vote_count','vote_average','popularity','genres']]
mov['year'] = pd.to_datetime(mov['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)


BUILDING CHARTS FOR TOP MOVIES BASED ON YEAR 

In [20]:
def top_movies_per_year(year,percentile,number):
    movie= mov[mov['year'] == year]
    average= movie[movie['vote_average'].notnull()]['vote_average']
    counts= movie[movie['vote_count'].notnull()]['vote_count']
    C = movie[movie['vote_average'].notnull()]['vote_average'].mean()
    m = counts.quantile(percentile)
    qualified = movie[(movie['vote_count'] >= m) & (movie['vote_count'].notnull()) & (movie['vote_average'].notnull())]
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified= qualified.sort_values('wr',ascending=False).head(number)

    return  qualified

In [21]:
#The top 50 movies in 2014 
top_movies_per_year(year='2014',percentile=0.90,number=50)

,movie_id,title,release_date,vote_count,vote_average,popularity,genres,year,wr
95,157336,Interstellar,05-11-2014,10867,8.1,724.247784,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 18, ""...",2014,7.510940
94,118340,Guardians of the Galaxy,30-07-2014,9742,7.9,481.098624,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...",2014,7.310939
2528,205596,The Imitation Game,14-11-2014,5723,8.0,145.364591,"[{""id"": 36, ""name"": ""History""}, {""id"": 18, ""na...",2014,7.112283
3871,244786,Whiplash,10-10-2014,4254,8.3,192.528841,"[{""id"": 18, ""name"": ""Drama""}]",2014,7.108601
693,210577,Gone Girl,01-10-2014,5862,7.9,143.041543,"[{""id"": 9648, ""name"": ""Mystery""}, {""id"": 53, ""...",2014,7.061843
88,177572,Big Hero 6,24-10-2014,6135,7.8,203.734590,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...",2014,7.021129
1538,120467,The Grand Budapest Hotel,26-02-2014,4519,8.0,74.417456,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",2014,6.975822
85,100402,Captain America: The Winter Soldier,20-03-2014,5764,7.6,72.225265,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2014,6.862299
46,127585,X-Men: Days of Future Past,15-05-2014,6032,7.5,118.078691,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",2014,6.818909
74,137113,Edge of Tomorrow,27-05-2014,4858,7.6,79.456485,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...",2014,6.780506


# CONTENT BASED RECOMMENDATION

In [22]:
cb_df =df[['movie_id','title','overview','genres','keywords','cast','crew']]

In [23]:
cb_df.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [24]:
cb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4808 entries, 0 to 4807
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4808 non-null   int64 
 1   title     4808 non-null   object
 2   overview  4805 non-null   object
 3   genres    4808 non-null   object
 4   keywords  4808 non-null   object
 5   cast      4808 non-null   object
 6   crew      4808 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.5+ KB


In [25]:
cb_df.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [26]:
cb_df.dropna(inplace=True)

In [27]:
cb_df.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [28]:
cb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4805 entries, 0 to 4807
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4805 non-null   int64 
 1   title     4805 non-null   object
 2   overview  4805 non-null   object
 3   genres    4805 non-null   object
 4   keywords  4805 non-null   object
 5   cast      4805 non-null   object
 6   crew      4805 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.3+ KB


In [29]:
cb_df.duplicated().sum()

0

In [30]:
cb_df['genres']=cb_df['genres'].replace("'", '\"')

In [31]:
cb_df['genres'].head()

0    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1    [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
Name: genres, dtype: object

In [32]:
def conv(x):
    L=[]
    for i in literal_eval(x):
        L.append(i['name'])
    return L

In [33]:
cb_df['keywords']=cb_df['keywords'].apply(conv)
cb_df['genres']=cb_df['genres'].apply(conv)

In [34]:
cb_df.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [35]:
def conv1(x):
    M=[]
    counter=0
    for i in literal_eval(x):
        if counter !=5 :
            M.append(i['name'])
            counter +=1
        else:
            break
        return M

In [36]:
cb_df['cast']=cb_df['cast'].apply(conv1)

In [37]:
def conv2(x):
    M=[]
    for i in literal_eval(x):
        if i['job']=='Director':
            M.append(i['name'])
        return M

In [38]:
cb_df['crew']=cb_df['crew'].apply(conv2)

In [39]:
cb_df['overview']=cb_df['overview'].apply(lambda x: x.split())

In [40]:
#closing spaces that may have been created

cb_df['keywords']=cb_df['keywords'].replace(' ','')
cb_df['cast']=cb_df['cast'].replace(' ','')
cb_df['genres']=cb_df['genres'].replace(' ','')
cb_df['crew']=cb_df['crew'].replace(' ','')

In [41]:
cb_df.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",[Sam Worthington],[]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",[Johnny Depp],[]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",[Daniel Craig],[]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",[Christian Bale],[]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",[Taylor Kitsch],[]


In [42]:
cb_df['tags']= cb_df['overview']+cb_df['genres']+cb_df['keywords']+cb_df['cast']+cb_df['crew']

In [43]:
cb_df.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",[Sam Worthington],[],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",[Johnny Depp],[],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",[Daniel Craig],[],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",[Christian Bale],[],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",[Taylor Kitsch],[],"[John, Carter, is, a, war-weary,, former, mili..."


In [44]:
new_cb_df= cb_df[['movie_id','title','tags']]
new_cb_df.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [45]:
new_cb_df['tags'].dtype

dtype('O')

In [46]:
#new_cb_df['tags'] = new_cb_df['tags'].apply(lambda x: [i.lower() for i in x])
new_cb_df['tags'] = new_cb_df['tags'].apply(lambda x: [i.lower() for i in x] if isinstance(x, list) else [])

In [47]:
new_cb_df['tags'].head(8)

0    [in, the, 22nd, century,, a, paraplegic, marin...
1    [captain, barbossa,, long, believed, to, be, d...
2    [a, cryptic, message, from, bond’s, past, send...
3    [following, the, death, of, district, attorney...
4    [john, carter, is, a, war-weary,, former, mili...
5    [the, seemingly, invincible, spider-man, goes,...
6    [when, the, kingdom's, most, wanted-and, most,...
7    [when, tony, stark, tries, to, jumpstart, a, d...
Name: tags, dtype: object

In [48]:
tf=TfidfVectorizer(max_features=1000, stop_words='english',min_df=0)

new_data = tf.fit_transform(new_cb_df['tags'].apply(lambda x: ' '.join(x)))

In [49]:
new_data.shape


(4805, 1000)

In [50]:
similarity= cosine_similarity(new_data)
    

In [51]:
new_cb_df[new_cb_df['title']=='The Dark Knight Rises'].index[0]

3

In [52]:
dist= sorted(list(enumerate(similarity[3])),reverse=True, key=lambda x:x[1])
for i in dist[0:6]:
    print(new_cb_df.iloc[i[0]].title)

The Dark Knight Rises
Batman
The Dark Knight
Batman
Batman Begins
Defendor


In [53]:
def recommend_movies(movie_name):
    idx=new_cb_df[new_cb_df['title']==movie_name].index[0]
    dist= sorted(list(enumerate(similarity[idx])),reverse=True, key=lambda x:x[1])
    
    for i in dist[0:6]:
        print(new_cb_df.iloc[i[0]].title)

In [54]:
recommend_movies('John Carter')

John Carter
Star Trek: Insurrection
Escape from Planet Earth
Planet 51
Avatar
Aliens


In [55]:
similarity.shape

(4805, 4805)

In [56]:
import pickle

In [57]:
pickle.dump(new_cb_df,open('movies_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [60]:
import streamlit as st

In [ ]:
pip install --upgrade streamlit
